<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220407_sm_SANUP_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers
# !pip install git+https://github.com/ssut/py-hanspell.git
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl (149.9 MB)
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-qwhmki2m
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-qwhmki2m
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- TPU 사용을 위한 accelerate

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path
# from hanspell import spell_checker
from pykospacing import Spacing

In [5]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [6]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [7]:
df = df.fillna('')

In [8]:
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,제품입고 워싱 청바지워싱,C 13 134
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의요청에의해 실내인테리어,F 42 424
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반소비자에게 여성의류 판매,G 47 474
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반고객을대상으로 필라테스,P 85 856


In [9]:
train = df[['text', 'target']]
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...
999995,제품입고 워싱 청바지워싱,C 13 134
999996,현장에서 고객의요청에의해 실내인테리어,F 42 424
999997,영업점에서 일반소비자에게 여성의류 판매,G 47 474
999998,사업장에서 일반고객을대상으로 필라테스,P 85 856


# 토크나이저, 함수

In [10]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [11]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [12]:
my_dict = {}
for i in range(len(train['target'].unique())):
  my_dict[train['target'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict[target]

train['target'] = train['target'].map(target_to_num)
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,0
1,상점내에서 일반인을 대상으로 채소.과일판매,1
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,2
3,영업점에서 일반소비자에게 열쇠잠금장치,3
4,어린이집 보호자의 위탁을 받아 취학전아동보육,4
...,...,...
999995,제품입고 워싱 청바지워싱,103
999996,현장에서 고객의요청에의해 실내인테리어,68
999997,영업점에서 일반소비자에게 여성의류 판매,11
999998,사업장에서 일반고객을대상으로 필라테스,12


In [13]:
def remove_something(text:str):
  return text.replace('&',' ')

train['text'] = train['text'].map(remove_something)

def remove_something(text:str):
  return text.replace('.',' ')

train['text'] = train['text'].map(remove_something)

def remove_something(text:str):
  return text.replace(',',' ')

train['text'] = train['text'].map(remove_something)

In [14]:
my_dict['  '] = len(my_dict)

In [15]:
my_dict

{'  ': 225,
 'A 1 11': 174,
 'A 1 12': 197,
 'A 1 14': 188,
 'A 2 20': 171,
 'A 3 31': 199,
 'A 3 32': 179,
 'B 5 51': 223,
 'B 6 61': 209,
 'B 6 62': 207,
 'B 7 71': 121,
 'B 7 72': 80,
 'B 8 80': 218,
 'C 10 101': 97,
 'C 10 102': 10,
 'C 10 103': 88,
 'C 10 104': 100,
 'C 10 105': 135,
 'C 10 106': 72,
 'C 10 107': 7,
 'C 10 108': 178,
 'C 11 111': 148,
 'C 11 112': 110,
 'C 12 120': 224,
 'C 13 131': 158,
 'C 13 132': 107,
 'C 13 133': 177,
 'C 13 134': 103,
 'C 13 139': 111,
 'C 14 141': 43,
 'C 14 142': 215,
 'C 14 143': 180,
 'C 14 144': 122,
 'C 15 151': 91,
 'C 15 152': 182,
 'C 16 161': 95,
 'C 16 162': 115,
 'C 16 163': 67,
 'C 17 171': 108,
 'C 17 172': 138,
 'C 17 179': 129,
 'C 18 181': 81,
 'C 18 182': 204,
 'C 19 191': 195,
 'C 19 192': 203,
 'C 20 201': 151,
 'C 20 202': 90,
 'C 20 203': 56,
 'C 20 204': 79,
 'C 20 205': 157,
 'C 21 211': 206,
 'C 21 212': 187,
 'C 21 213': 201,
 'C 22 221': 18,
 'C 22 222': 33,
 'C 23 231': 156,
 'C 23 232': 120,
 'C 23 233': 94,
 'C 

In [16]:
train[train['text'].isnull()]

,text,target


## 띄어쓰기

12시간 걸림

In [ ]:
train20 = train[:200000]
train40 = train[200000:400000]
train60 = train[400000:600000]
train80 = train[600000:800000]
train100 = train[800000:]

In [ ]:
for df in dfs:
  

In [ ]:
spacing = Spacing()

def text_space(text:str):
  return spacing(text)

train['text'] = train['text'].map(text_space)

In [ ]:
train

In [ ]:
train.to_csv('/content/drive/MyDrive/220406_inference/train.csv', index=False)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [ ]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32)
  # eval_dataloader = torch.utils.data.DataLoader(
  #                   train_dataset,
  #                   batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader = accelerator.prepare(model, optimizer, train_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220406_inference/model', save_function=accelerator.save)

In [ ]:
print('--------------------------------')

tokenized_train = tokenizer(
  list(train['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
train_label = train['target']

train_dataset = BERTDataset(tokenized_train, train_label)

notebook_launcher(training_function)

# 추론

In [ ]:
test = test.fillna('')

In [ ]:
test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
test

In [ ]:
test = test[['text', 'target']]
test

In [ ]:
def text_space(text:str):
  return spacing(text)

test['text'] = test['text'].map(text_space)

In [ ]:
test.to_csv('/content/drive/MyDrive/220406_inference/test.csv', index=False)

In [ ]:
test['target'].loc[0]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = test['target'].map(target_to_num)

tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220406_inference/model', num_labels=225)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob = pd.DataFrame(output_prob)

  df_label.to_csv('/content/drive/MyDrive/220406_inference/pred_label.csv', index=False)
  df_prob.to_csv('/content/drive/MyDrive/220406_inference/pred_prob.csv', index=False)

In [ ]:
print(my_variable)

In [ ]:
submission

In [ ]:
pred0

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label4.csv')

In [ ]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']
df = test

In [ ]:
df.head(3)

In [ ]:
1-((len(df)-len(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])]))/len(df))

In [ ]:
hard = df[df.merge(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])], how = 'left', left_index=True, right_index=True)['text_y'].isnull()]
hard

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob4.csv')

In [ ]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test = pd.concat([test, pred], axis=1)
test.head(3)

In [ ]:
def num_to_label(label):
    label_dict = label_dict = {v: k for k, v in my_dict.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

test['soft'] = pd.DataFrame(answer)[1]
test.head(3)

In [ ]:
soft = test[['label0', 'label1', 'label2', 'label3', 'label4', 'soft']]
soft

In [ ]:
def label_count(df):
  num_count = {}
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    try: num_count[df[col]] += 1
    except: num_count[df[col]] = 1
  return(num_count)

temp = soft.apply(label_count, axis=1)
soft['temp'] = temp

def label_count(temp:dict):
  keys = []
  temptemp = 0
  for k, v in temp.items():
    if v >= 3:
      temptemp = k
  keys.append(temptemp)
  return keys[0]
  
soft['hard'] = soft['temp'].map(label_count)

In [ ]:
soft[soft['soft'] != soft['hard']]

In [ ]:
df = soft[['label0',	'label1',	'label2',	'label3',	'label4',	'soft',	'hard']]
df

In [ ]:
submission

In [ ]:
print(my_variable)

In [ ]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = df['soft'].map(split_label)

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220404_inference/submission.csv', index=False)